In [82]:
from sklearn.linear_model import Ridge, Lasso, LogisticRegression
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

# Load the tables 
covid_df = pd.read_csv('../data/covid.csv', index_col=False)
precovid_df = pd.read_csv('../data/precovid.csv', index_col=False)
postcovid_df = pd.read_csv('../data/postcovid.csv', index_col=False)

### Lasso Regularization embedded feature selection

In [83]:
def get_embedded_report(inp_df, estimator, count, regularizer):
    
    input_df = inp_df.drop(columns={'Number of Workers','percent_layoff','Unnamed: 0'})
    X = input_df
    y = inp_df['Number of Workers']
    
    # First normalize our predictors
    X_numeric_scaled = StandardScaler().fit_transform(X)
    y_scaled     = StandardScaler().fit_transform(y.values.reshape(-1,1))

    # Save the columns name into variable named 'features'
    features = X.columns

    model = estimator.fit(X, y_scaled)
    try:
        c = model.coef_
    except:
        c = model.feature_importances_
    finally:
        if regularizer=='lasso':
            output_df = pd.DataFrame(list(zip(features,c)), columns=['Features','Coefficient'])
            return output_df[output_df['Coefficient']>0].sort_values(by=['Coefficient'], ascending=False).reset_index(drop=True).head(count)
        else:
            output_df = pd.DataFrame(list(zip(features,c.flatten())), columns=['Features','Coefficient'])
            return output_df.sort_values(by=['Coefficient'], ascending=False).reset_index(drop=True).head(count)


In [84]:
count = 20
precovid_features = get_embedded_report(precovid_df, Lasso(alpha=0.1), count, 'lasso')
covid_features = get_embedded_report(covid_df, Lasso(alpha=0.1), count, 'lasso')
postcovid_features = get_embedded_report(postcovid_df, Lasso(alpha=0.1), count, 'lasso')

/Users/Bhavya/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.184e+02, tolerance: 3.430e-02
  model = cd_fast.enet_coordinate_descent(
/Users/Bhavya/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.037e+01, tolerance: 2.690e-02
  model = cd_fast.enet_coordinate_descent(
/Users/Bhavya/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing reg

In [85]:
print(precovid_features)
precovid_df[precovid_features['Features']]

                             Features   Coefficient
0                   Number of Layoffs  5.438326e-02
1                   industry_labelled  6.643478e-04
2                      employee_count  4.274054e-06
3                  capitalExpenditure  8.818919e-10
4                   commonStockIssued  7.899566e-10
5      ResearchAndDevelopmentExpenses  6.368232e-10
6                           netIncome  5.196202e-10
7                         grossProfit  4.266340e-10
8         depreciationAndAmortization  3.264021e-10
9          effectOfForexChangesOnCash  3.228086e-10
10                      otherExpenses  2.354769e-10
11        SellingAndMarketingExpenses  1.663071e-10
12      inventory_cash-flow-statement  1.558306e-10
13                   incomeTaxExpense  1.503780e-10
14              weightedAverageShsOut  1.498765e-10
15             commonStockRepurchased  8.939251e-11
16   GeneralAndAdministrativeExpenses  7.370204e-11
17          deferredRevenueNonCurrent  6.703442e-11
18          

,Number of Layoffs,industry_labelled,employee_count,capitalExpenditure,commonStockIssued,ResearchAndDevelopmentExpenses,netIncome,grossProfit,depreciationAndAmortization,effectOfForexChangesOnCash,otherExpenses,SellingAndMarketingExpenses,inventory_cash-flow-statement,incomeTaxExpense,weightedAverageShsOut,commonStockRepurchased,GeneralAndAdministrativeExpenses,deferredRevenueNonCurrent,freeCashFlow,deferrredTaxLiabilitiesNonCurrent
0,1,2,118033.0,-1.018000e+09,0.0,0.000000e+00,6.620000e+08,2.946000e+09,0.000000e+00,0.0,1.271000e+09,401000000.0,0.000000e+00,2.200000e+08,4.450080e+08,-1.700000e+07,0.0,5.249000e+09,-2.820000e+08,0.0
1,1,4,132000.0,-3.267000e+09,1000000.0,3.701000e+09,1.151900e+10,2.042100e+10,2.665000e+09,0.0,0.000000e+00,0.0,1.693000e+09,1.765000e+09,1.952867e+10,-2.078300e+10,0.0,2.878000e+09,1.122100e+10,398000000.0
2,1,4,137000.0,-2.363000e+09,0.0,3.948000e+09,1.156100e+10,2.182100e+10,3.040000e+09,0.0,0.000000e+00,0.0,7.000000e+07,2.232000e+09,1.869628e+10,-2.370200e+10,0.0,0.000000e+00,8.792000e+09,0.0
3,1,5,3641.0,0.000000e+00,0.0,0.000000e+00,5.988000e+07,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,6.879000e+06,9.541878e+07,0.000000e+00,0.0,0.000000e+00,6.597500e+07,0.0
4,1,11,6700.0,-8.000000e+06,0.0,0.000000e+00,1.040000e+07,1.275000e+08,1.830000e+07,400000.0,9.300000e+06,0.0,-4.350000e+07,-4.300000e+06,1.068000e+08,-9.100000e+06,0.0,0.000000e+00,5.240000e+07,174800000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,1,568,7700.0,-2.650000e+07,0.0,1.010000e+07,5.520000e+07,1.356000e+08,2.620000e+07,-200000.0,-2.700000e+06,0.0,0.000000e+00,8.000000e+06,7.390000e+07,0.000000e+00,64900000.0,0.000000e+00,6.190000e+07,11700000.0
339,2,570,11500.0,-1.850000e+07,0.0,0.000000e+00,1.350000e+08,5.383000e+08,6.110000e+07,0.0,0.000000e+00,0.0,0.000000e+00,2.720000e+07,4.233000e+08,-1.422000e+08,0.0,0.000000e+00,2.442000e+08,161300000.0
340,1,570,11500.0,-3.420000e+07,0.0,0.000000e+00,1.354000e+08,5.512000e+08,6.700000e+07,0.0,0.000000e+00,0.0,0.000000e+00,6.210000e+07,4.195000e+08,-6.880000e+07,0.0,0.000000e+00,2.151000e+08,152100000.0
341,1,571,9000.0,-2.356400e+07,9565000.0,4.066100e+07,6.610700e+07,1.894890e+08,0.000000e+00,1081000.0,0.000000e+00,0.0,0.000000e+00,2.620700e+07,6.194100e+07,-7.126200e+07,52980000.0,3.329270e+08,5.468400e+07,173289000.0


In [86]:
print(covid_features)
covid_df[covid_features['Features']]


                                           Features   Coefficient
0                              incomeBeforeTaxRatio  1.781151e-02
1                                        new_deaths  1.038040e-03
2                              operatingIncomeRatio  1.024596e-03
3                                 industry_labelled  1.210111e-04
4                                         new_cases  1.783678e-05
5                                    employee_count  4.629144e-07
6                            stockBasedCompensation  5.271508e-09
7                                   interestExpense  4.937152e-09
8                     netIncome_cash-flow-statement  2.929060e-09
9                                  incomeTaxExpense  2.011775e-09
10                         weightedAverageShsOutDil  1.928979e-09
11                               capitalExpenditure  1.484089e-09
12                      depreciationAndAmortization  1.446167e-09
13                                     freeCashFlow  7.539257e-10
14        

,incomeBeforeTaxRatio,new_deaths,operatingIncomeRatio,industry_labelled,new_cases,employee_count,stockBasedCompensation,interestExpense,netIncome_cash-flow-statement,incomeTaxExpense,weightedAverageShsOutDil,capitalExpenditure,depreciationAndAmortization,freeCashFlow,deferredIncomeTax,ResearchAndDevelopmentExpenses,depreciationAndAmortization_cash-flow-statement,dividendsPaid,inventory_cash-flow-statement,accountsPayables
0,-0.975418,890.315217,-0.904822,2,50113.815217,118033.0,0.0,340000000.0,0.000000e+00,-6.960000e+08,5.090490e+08,-5.770000e+08,0.000000e+00,-3.181000e+09,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00
1,-0.392465,2236.066667,-0.328094,2,116989.855556,118033.0,0.0,371000000.0,0.000000e+00,-3.230000e+08,6.346090e+08,0.000000e+00,0.000000e+00,1.740000e+08,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00
2,0.100960,1372.780220,0.105080,3,26741.428571,0.0,10029000.0,13422000.0,1.899600e+08,6.258000e+07,6.929400e+07,-5.713200e+07,5.664200e+07,3.801700e+08,4435000.0,0.000000e+00,5.664200e+07,-1.742400e+07,172793000.0,3.599000e+06
3,0.257147,1035.130435,0.300237,6,105140.771739,47000.0,135000000.0,596000000.0,3.179000e+09,5.080000e+08,1.777000e+09,-2.170000e+08,2.127000e+09,7.718000e+09,-34000000.0,1.673000e+09,2.127000e+09,-2.315000e+09,219000000.0,1.146000e+09
4,0.045879,1372.780220,0.056882,8,26741.428571,7600.0,3100000.0,15900000.0,4.960000e+07,1.670000e+07,1.930000e+07,-9.100000e+06,9.700000e+06,4.178000e+08,0.0,0.000000e+00,9.700000e+06,0.000000e+00,455600000.0,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,0.085937,1372.780220,0.069550,571,26741.428571,7100.0,6613000.0,8737000.0,3.846500e+07,6.551000e+06,6.342700e+07,-9.711000e+06,3.210600e+07,1.505260e+08,-1891000.0,3.452200e+07,3.210600e+07,-5.062000e+06,22580000.0,0.000000e+00
265,-0.284075,1372.780220,-0.193209,573,26741.428571,22514.0,0.0,64000000.0,-5.890000e+08,-5.000000e+06,1.753270e+08,-1.730000e+08,0.000000e+00,-3.930000e+08,-18000000.0,0.000000e+00,0.000000e+00,-1.000000e+06,448000000.0,0.000000e+00
266,-0.581858,1541.793478,-0.583956,574,134353.271739,0.0,0.0,313000000.0,-2.007000e+10,-6.010000e+09,4.272000e+09,-3.629000e+09,4.599328e+10,3.760000e+08,0.0,0.000000e+00,0.000000e+00,-3.715000e+09,0.0,0.000000e+00
267,-0.244395,1372.780220,-0.212597,576,26741.428571,5100.0,10200000.0,12300000.0,-9.540000e+07,-2.400000e+07,2.187000e+08,-1.320000e+07,7.890000e+07,1.619000e+08,-24300000.0,0.000000e+00,7.890000e+07,-2.190000e+07,56300000.0,-5.980000e+07


In [87]:
print(postcovid_features)
postcovid_df[postcovid_features['Features']]


                            Features   Coefficient
0               operatingIncomeRatio  3.429333e-03
1                  industry_labelled  1.889191e-03
2                         new_deaths  6.059916e-04
3                     employee_count  8.691729e-06
4             stockBasedCompensation  2.214297e-08
5                  commonStockIssued  1.153897e-08
6                   incomeTaxExpense  3.342457e-09
7      inventory_cash-flow-statement  2.853732e-09
8         effectOfForexChangesOnCash  2.724566e-09
9                   accountsPayables  1.866075e-09
10       depreciationAndAmortization  1.801349e-09
11               otherWorkingCapital  1.143202e-09
12                       taxPayables  8.728512e-10
13                     debtRepayment  7.004915e-10
14             weightedAverageShsOut  6.704592e-10
15                     otherExpenses  5.787594e-10
16  netCashUsedForInvestingActivites  5.040531e-10
17                 otherNonCashItems  4.150778e-10
18           otherFinancingActi

,operatingIncomeRatio,industry_labelled,new_deaths,employee_count,stockBasedCompensation,commonStockIssued,incomeTaxExpense,inventory_cash-flow-statement,effectOfForexChangesOnCash,accountsPayables,depreciationAndAmortization,otherWorkingCapital,taxPayables,debtRepayment,weightedAverageShsOut,otherExpenses,netCashUsedForInvestingActivites,otherNonCashItems,otherFinancingActivites,ResearchAndDevelopmentExpenses
0,0.000000,2,373.391304,118033.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00
1,0.348427,6,1783.786517,47000.0,306000000.0,0.0,4.360000e+08,-3.850000e+08,7000000.0,-2.580000e+08,2.053000e+09,438000000.0,0.0,-4.879000e+09,1.771000e+09,0.000000e+00,-1.591000e+09,-472000000.0,1.903000e+09,1.497000e+09
2,0.310762,6,430.739130,47000.0,126000000.0,0.0,4.480000e+08,-1.700000e+07,-59000000.0,7.780000e+08,2.205000e+09,294000000.0,0.0,-2.701000e+09,1.771000e+09,2.290000e+08,1.286000e+09,602000000.0,-3.090000e+08,1.614000e+09
3,0.073506,9,1335.913043,114000.0,8500000.0,0.0,2.720000e+07,0.000000e+00,1900000.0,-3.000000e+06,2.230000e+07,-4600000.0,146000000.0,-3.330000e+07,6.720000e+07,1.260200e+09,-5.200000e+06,1700000.0,-1.180000e+07,0.000000e+00
4,0.039332,9,430.739130,114000.0,6100000.0,0.0,1.490000e+07,0.000000e+00,-1000000.0,0.000000e+00,3.000000e+07,0.0,124700000.0,-4.222000e+08,6.640000e+07,1.593400e+09,-1.300000e+07,-112900000.0,4.994000e+08,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,0.177044,562,363.747253,48250.0,26000000.0,0.0,1.890000e+08,0.000000e+00,0.0,0.000000e+00,5.080000e+08,-52000000.0,0.0,-2.212000e+09,4.144000e+08,5.220000e+08,-5.280000e+08,52000000.0,2.979000e+09,0.000000e+00
100,0.037565,563,1783.786517,2200000.0,0.0,0.0,7.980000e+08,-4.699000e+09,49000000.0,-1.640000e+09,2.680000e+09,0.0,904000000.0,-9.260000e+08,2.769000e+09,0.000000e+00,-4.558000e+09,49000000.0,1.019200e+10,0.000000e+00
101,0.044839,563,363.747253,2200000.0,0.0,0.0,1.497000e+09,9.690000e+08,-149000000.0,1.187000e+09,2.699000e+09,0.0,868000000.0,-5.130000e+08,2.736000e+09,0.000000e+00,-4.026000e+09,302000000.0,-1.325000e+09,0.000000e+00
102,0.010516,566,1783.786517,49300.0,24500000.0,0.0,1.800000e+06,-1.590000e+07,14300000.0,5.890000e+07,3.736000e+08,41000000.0,0.0,-4.040000e+08,2.634000e+08,8.810000e+07,-1.518000e+08,643400000.0,5.056000e+08,0.000000e+00


### Ridge Regularization Embedded Feature Seelection

In [88]:
# Using L2-regularization
precovid_ridge_features = get_embedded_report(precovid_df, Ridge(), count, 'ridge')
covid_ridge_features = get_embedded_report(covid_df, Ridge(), count, 'ridge')
postcovid_ridge_features = get_embedded_report(postcovid_df, Ridge(), count, 'ridge')

/Users/Bhavya/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(


In [89]:
print(precovid_ridge_features)
precovid_df[precovid_ridge_features['Features']]

                                    Features   Coefficient
0                             netIncomeRatio  5.656870e-01
1                       operatingIncomeRatio  4.750188e-01
2                          Number of Layoffs  3.785652e-01
3                                       Year  1.446932e-01
4                                    Quarter  3.857612e-02
5                                        EPS  3.582766e-02
6        netCashProvidedByOperatingActivites  1.022301e-03
7                                    netDebt  9.259626e-04
8   netCashUsedProvidedByFinancingActivities  8.965106e-04
9      totalLiabilitiesAndStockholdersEquity  6.004911e-04
10                         industry_labelled  5.720195e-04
11                    cashAndCashEquivalents  5.634364e-04
12               cashAndShortTermInvestments  3.625263e-04
13                         cashAtEndOfPeriod  2.463377e-04
14          netCashUsedForInvestingActivites  2.090666e-05
15                            employee_count  2.157378e-

,netIncomeRatio,operatingIncomeRatio,Number of Layoffs,Year,Quarter,EPS,netCashProvidedByOperatingActivites,netDebt,netCashUsedProvidedByFinancingActivities,totalLiabilitiesAndStockholdersEquity,industry_labelled,cashAndCashEquivalents,cashAndShortTermInvestments,cashAtEndOfPeriod,netCashUsedForInvestingActivites,employee_count,EBITDA,goodwillAndIntangibleAssets,weightedAverageShsOut,freeCashFlow
0,0.055351,0.096405,1,2019,2,1.490000,7.360000e+08,3.442900e+10,1.179000e+09,6.196700e+10,2,3.190000e+08,5.407000e+09,3.300000e+08,-1.933000e+09,118033.0,1.157000e+09,6.196000e+09,4.450080e+08,-2.820000e+08
1,0.216258,0.236778,1,2018,2,0.590000,1.448800e+10,8.262900e+10,-3.152300e+10,3.491970e+11,4,3.197100e+10,7.097000e+10,3.197100e+10,3.947000e+09,132000.0,1.679500e+10,0.000000e+00,1.952867e+10,1.122100e+10
2,0.199276,0.231233,1,2019,1,0.617500,1.115500e+10,7.464200e+10,-2.945700e+10,3.419980e+11,4,3.798800e+10,8.009200e+10,3.981700e+10,1.334800e+10,137000.0,1.784300e+10,0.000000e+00,1.869628e+10,8.792000e+09
3,0.896958,0.000000,1,2018,4,0.627539,6.597500e+07,-9.393900e+07,-6.018800e+07,1.490701e+09,5,6.402060e+08,6.402060e+08,0.000000e+00,-5.787000e+06,3641.0,6.675900e+07,0.000000e+00,9.541878e+07,6.597500e+07
4,0.025628,0.028832,1,2018,1,0.100000,6.040000e+07,8.284000e+08,-7.000000e+06,2.754700e+09,11,1.227000e+08,1.227000e+08,1.227000e+08,-8.000000e+06,6700.0,3.380000e+07,1.506000e+09,1.068000e+08,5.240000e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,0.127867,0.140839,1,2018,3,0.750000,8.840000e+07,-1.011000e+08,1.010000e+07,1.946100e+09,568,2.973000e+08,2.973000e+08,2.973000e+08,-2.650000e+07,7700.0,9.140000e+07,1.273000e+08,7.390000e+07,6.190000e+07
339,0.103298,0.151044,2,2019,3,0.320000,2.627000e+08,1.366700e+09,-5.200000e+07,8.803700e+09,570,1.881300e+09,3.485600e+09,1.398200e+09,-3.000000e+07,11500.0,2.595000e+08,3.081100e+09,4.233000e+08,2.442000e+08
340,0.103541,0.173205,1,2019,4,0.320000,2.493000e+08,1.652900e+09,-1.660000e+08,8.758500e+09,570,1.818700e+09,3.517100e+09,1.456800e+09,-2.470000e+07,11500.0,3.020000e+08,3.061500e+09,4.195000e+08,2.151000e+08
341,0.087908,0.127457,1,2019,2,1.070000,7.824800e+07,1.127845e+09,-4.628500e+07,4.015603e+09,571,6.330200e+07,6.330200e+07,6.330200e+07,-3.504500e+07,9000.0,1.031120e+08,1.449349e+09,6.194100e+07,5.468400e+07


In [90]:
print(covid_ridge_features)
covid_df[covid_ridge_features['Features']]

                                    Features  Coefficient
0                       incomeBeforeTaxRatio     0.165152
1                                        EPS     0.142897
2                                    Quarter     0.131268
3                          Number of Layoffs     0.074990
4                                 new_deaths     0.014892
5                                  totalDebt     0.000318
6                         new_cases_smoothed     0.000244
7                    totalStockholdersEquity     0.000236
8                     new_deaths_per_million     0.000202
9                 totalNonCurrentLiabilities     0.000190
10                          otherLiabilities     0.000190
11                   totalCurrentLiabilities     0.000189
12                   cashAtBeginningOfPeriod     0.000107
13                           netChangeInCash     0.000107
14               cashAndShortTermInvestments     0.000099
15                         operatingCashFlow     0.000093
16  netCashUse

,incomeBeforeTaxRatio,EPS,Quarter,Number of Layoffs,new_deaths,totalDebt,new_cases_smoothed,totalStockholdersEquity,new_deaths_per_million,totalNonCurrentLiabilities,otherLiabilities,totalCurrentLiabilities,cashAtBeginningOfPeriod,netChangeInCash,cashAndShortTermInvestments,operatingCashFlow,netCashUsedProvidedByFinancingActivities,new_cases_per_million,totalLiabilities,netCashProvidedByOperatingActivites
0,-0.975418,-4.710000,3,2,890.315217,4.120500e+10,50152.321380,-5.528000e+09,2.631826,5.171700e+10,0.0,1.658400e+10,4.670000e+08,-1.700000e+08,8.284000e+09,-2.604000e+09,1.511000e+09,148.138696,6.830100e+10,-2.604000e+09
1,-0.392465,-1.970000,1,3,2236.066667,4.802400e+10,120848.299989,-7.945000e+09,6.609944,5.926100e+10,0.0,1.733300e+10,3.990000e+08,3.500000e+07,1.403900e+10,1.740000e+08,7.013000e+09,345.827267,7.659400e+10,1.740000e+08
2,0.100960,2.750000,2,1,1372.780220,3.281740e+09,26039.981154,3.726519e+09,4.058055,3.766587e+09,0.0,4.437559e+09,1.279838e+09,-1.383360e+08,1.141502e+09,4.373020e+08,-5.191350e+08,79.048879,8.204146e+09,4.373020e+08
3,0.257147,1.798273,3,1,1035.130435,8.072100e+10,102092.024870,1.355000e+10,3.059891,1.067480e+11,0.0,2.853300e+10,8.546000e+09,3.636000e+09,1.224900e+10,7.935000e+09,-3.643000e+09,310.800826,1.352810e+11,7.935000e+09
4,0.045879,2.580000,2,2,1372.780220,1.853700e+09,26039.981154,7.131000e+08,4.058055,1.328100e+09,0.0,8.932000e+08,3.886000e+08,2.246000e+08,6.132000e+08,4.269000e+08,-1.932000e+08,79.048879,2.221300e+09,4.269000e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,0.085937,0.620000,2,1,1372.780220,9.294470e+08,26039.981154,1.909766e+09,4.058055,1.461275e+09,0.0,5.356280e+08,1.028490e+08,-1.486000e+06,9.848300e+07,1.602370e+08,-1.478450e+08,79.048879,1.996903e+09,1.602370e+08
265,-0.284075,-3.360000,2,1,1372.780220,5.657000e+09,26039.981154,3.617000e+09,4.058055,6.756000e+09,0.0,2.141000e+09,1.497000e+09,9.320000e+08,2.300000e+09,-2.200000e+08,1.323000e+09,79.048879,8.897000e+09,-2.200000e+08
266,-0.581858,-4.700000,4,1,1541.793478,6.596000e+10,129908.336978,1.571500e+11,4.557598,1.122570e+11,0.0,5.636300e+10,8.832000e+09,-4.468000e+09,4.787900e+10,4.005000e+09,-5.283000e+09,397.154261,1.686200e+11,4.005000e+09
267,-0.244395,-0.440000,2,1,1372.780220,2.183000e+09,26039.981154,4.643100e+09,4.058055,3.195800e+09,0.0,7.630000e+08,2.359000e+08,8.732000e+08,1.109100e+09,1.751000e+08,6.581000e+08,79.048879,3.958800e+09,1.751000e+08


In [91]:
print(postcovid_ridge_features)
postcovid_df[postcovid_ridge_features['Features']]

                                           Features   Coefficient
0                             weightedAverageShsOut  1.381479e-07
1                                         totalDebt  1.381093e-07
2                                   interestExpense  1.186155e-07
3                  netCashUsedForInvestingActivites  1.017495e-07
4                     netIncome_cash-flow-statement  8.595588e-08
5                            stockBasedCompensation  8.460799e-08
6                                     shortTermDebt  7.152236e-08
7                                  incomeTaxExpense  7.060667e-08
8                       depreciationAndAmortization  6.785777e-08
9   depreciationAndAmortization_cash-flow-statement  6.678758e-08
10                                     longTermDebt  6.658689e-08
11                               capitalExpenditure  6.034217e-08
12                      totalOtherIncomeExpensesNet  5.854689e-08
13                                otherNonCashItems  4.341635e-08
14        

,weightedAverageShsOut,totalDebt,interestExpense,netCashUsedForInvestingActivites,netIncome_cash-flow-statement,stockBasedCompensation,shortTermDebt,incomeTaxExpense,depreciationAndAmortization,depreciationAndAmortization_cash-flow-statement,longTermDebt,capitalExpenditure,totalOtherIncomeExpensesNet,otherNonCashItems,operatingIncome,changeInWorkingCapital,deferredIncomeTax,netIncome,revenue,netCashProvidedByOperatingActivites
0,0.000000e+00,4.368700e+10,0.0,0.000000e+00,0.000000e+00,0.0,4.739000e+09,0.000000e+00,0.000000e+00,0.000000e+00,3.894800e+10,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00
1,1.771000e+09,7.347400e+10,548000000.0,-1.591000e+09,4.490000e+09,306000000.0,9.952000e+09,4.360000e+08,2.053000e+09,2.053000e+09,6.352200e+10,-1.620000e+08,2.120000e+08,-472000000.0,4.717000e+09,-1.275000e+09,-194000000.0,4.490000e+09,1.353800e+10,4.908000e+09
2,1.771000e+09,6.960600e+10,560000000.0,1.286000e+09,3.952000e+09,126000000.0,9.207000e+09,4.480000e+08,2.205000e+09,2.205000e+09,6.039900e+10,-1.770000e+08,-2.030000e+08,602000000.0,4.603000e+09,1.338000e+09,-621000000.0,3.949000e+09,1.481200e+10,7.602000e+09
3,6.720000e+07,8.499000e+08,8500000.0,-5.200000e+06,7.460000e+07,8500000.0,1.508000e+08,2.720000e+07,2.230000e+07,2.230000e+07,6.991000e+08,-6.600000e+06,-7.800000e+06,1700000.0,1.097000e+08,-5.440000e+07,-7400000.0,7.460000e+07,1.492400e+09,4.530000e+07
4,6.640000e+07,1.402600e+09,16000000.0,-1.300000e+07,4.880000e+07,6100000.0,2.118000e+08,1.490000e+07,3.000000e+07,3.000000e+07,1.190800e+09,-1.300000e+07,-1.540000e+07,-112900000.0,7.910000e+07,1.171000e+08,28000000.0,4.890000e+07,2.011100e+09,1.171000e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,4.144000e+08,1.427700e+10,93000000.0,-5.280000e+08,5.870000e+08,26000000.0,2.310000e+08,1.890000e+08,5.080000e+08,5.080000e+08,1.404600e+10,-5.500000e+08,-1.140000e+08,52000000.0,8.900000e+08,-7.300000e+07,-53000000.0,5.870000e+08,5.027000e+09,1.047000e+09
100,2.769000e+09,6.681700e+10,0.0,-4.558000e+09,2.054000e+09,0.0,1.700800e+10,7.980000e+08,2.680000e+09,2.680000e+09,4.980900e+10,-3.539000e+09,-2.417000e+09,49000000.0,5.318000e+09,-8.472000e+09,-69000000.0,2.054000e+09,1.415690e+11,-3.758000e+09
101,2.736000e+09,6.530900e+10,479000000.0,-4.026000e+09,5.147000e+09,0.0,1.794800e+10,1.497000e+09,2.699000e+09,2.699000e+09,4.736100e+10,-3.953000e+09,-2.100000e+08,302000000.0,6.854000e+09,4.670000e+09,180000000.0,5.149000e+09,1.528590e+11,1.299800e+10
102,2.634000e+08,8.374000e+09,84900000.0,-1.518000e+08,3.990000e+07,24500000.0,4.196000e+08,1.800000e+06,3.736000e+08,0.000000e+00,7.954400e+09,-1.810000e+08,-1.410000e+07,643400000.0,5.660000e+07,-2.319000e+08,-86000000.0,3.990000e+07,5.382100e+09,3.899000e+08
